In [ ]:
import os
import sys

current_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(current_dir, '..', 'module'))
results_path = os.path.abspath(os.path.join(current_dir, '..', 'results'))
sys.path.append(module_path)
file_name = 'lth_test.xlsx'
##########################  Target init  #############################

from target import target_init
from DSview_ctrl import pos_init
import argparse

ret_list = target_init()

IDF_PATH = ret_list[0]
EXAMPLE_PATH = ret_list[1]
target_port = ret_list[2]

input_pos = pos_init()

parser = argparse.ArgumentParser(description='命令行中传入一个xlsx文件名')
parser.add_argument('--file_name', type=str, required=True, help='传入的文件名')
# parser.add_argument('--except_num', type=int, required=True, help='取第n位最大最小值')
args = parser.parse_args((results_path + ' ' + file_name).split())

In [ ]:
####### Traversing different DFS ,PD_flash and times configurations then build and download  ########
from openpyxl import Workbook
from main_func import main_func
import pprint
from draw_html import draw_html_for_results_plotly

wb = Workbook()
DFS_min_list = [8, 16, 32]
DFS_max_list = [48, 64, 96]
PD_flash_flag_list = ['PU_flash', 'PD_flash']
RTC_source_list = ['internal_136k_Rc', 'internal_32k_Rc', 'external_32k_crystal']
times_list = [1]
optional_items_list = ['CPU register restore', 'CPU frequency restore',\
                      'Calc sleep_time_overhead_out', 'DFS frequency up',\
                      'Calc sleep_time_overhead_in',\
                      'from overhead_in to enter light_sleep','CPU register save',\
                      'sw sleep_time_overhead_out','sw sleep_time_overhead_in',\
                      'dev_sleep_time_overhead_out','dev_sleep_time_overhead_in']
csv_sublist = optional_items_list[:7]
serial_sublist = optional_items_list[7:9]
dev_sublist = [optional_items_list[2], optional_items_list[-4], optional_items_list[-2],\
              optional_items_list[4], optional_items_list[-3], optional_items_list[-1]]

####### main_func will set up the menu in wb according to list gived, and then #######
####### first, change example menuconfig ##########
####### secondly, change sleep time interval (if need), download program to chip, and let it enter download mode ########
####### thirdly, control mouse to use DSview , Wait for trigger ########
####### fourthly, mannually select serial to reset chip ########
####### fifthly, process serial data and record it in wb, serial send data sequence should be equivalent to serial_sublist ########
####### sixthly, start to export csv, sampling rate should be 1 M ########
####### seventhly, Grab the part of the screen to make sure sucessfully export csv ########
####### eighthly, process csv and record it in wb ########
####### ninthly, calc dev and record it in wb ########
####### tenthly, build dict_list for chart ########
dict_list = []
offset = [0, 0]
csvname = '/home/loutianhao/Downloads/1.csv'
main_func(DFS_min_list, DFS_max_list, RTC_source_list, times_list, PD_flash_flag_list, csvname, wb, IDF_PATH, EXAMPLE_PATH, \
                 csv_sublist, serial_sublist, dev_sublist, optional_items_list,  \
                 input_pos, target_port, offset, dict_list, baud = 115200)

maximum_csv_sublist = ['maximum ' + s for s in csv_sublist]
minimum_csv_sublist = ['minimum ' + s for s in csv_sublist]
draw_html_for_results_plotly(dict_list, {'csv_maximum': maximum_csv_sublist, 'csv_minimum': minimum_csv_sublist}, \
                          DFS_min_list, DFS_max_list, RTC_source_list, times_list, PD_flash_flag_list, "CSV_")

dev_sublist_for_draw = dev_sublist[2::3]
maximum_dev_sublist = ['maximum ' + s for s in dev_sublist_for_draw]
minimum_dev_sublist = ['minimum ' + s for s in dev_sublist_for_draw]
draw_html_for_results_plotly(dict_list, {'dev_maximum': maximum_dev_sublist, 'dev_minimum': minimum_dev_sublist}, \
                          DFS_min_list, DFS_max_list, RTC_source_list, times_list, PD_flash_flag_list, "DEV_")

wb.save(args.file_name)

with open('../results/dict_list.txt', 'w', encoding='utf-8') as file:
    pprint.pprint(dict_list, stream=file)

print("process done")


In [ ]:
import ast

# 从 'dict_list.txt' 文件中读取数据
with open('dict_list.txt', 'r', encoding='utf-8') as file:
    data_str = file.read()

# 将字符串转换回 Python 对象（dict_list）
dict_list = ast.literal_eval(data_str)

# 现在可以使用 dict_list 变量进行后续操作
print(dict_list)